# Paper Scraping

In [1]:
import os
import requests

In [2]:
headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'}

Insert here the url of the desired page of Google Scholar to scrape

In [3]:
# url = 'https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5&q=object+detection+in+aerial+image+&btnG=&oq=ob'
url = "https://scholar.google.com/scholar?hl=it&as_sdt=0%2C5&q=face+recognition+system&oq=face+reco"

In [4]:
response=requests.get(url,headers=headers)

In [5]:
from bs4 import BeautifulSoup

In [6]:
# this function for the getting inforamtion of the web page
def get_paperinfo(paper_url):

  #download the page
    response=requests.get(url,headers=headers)

  # check successful response
    if response.status_code != 200:
        print('Status code:', response.status_code)
        raise Exception('Failed to fetch web page ')

  #parse using beautiful soup
    paper_doc = BeautifulSoup(response.text,'html.parser')

    return paper_doc

For extracting tags , we will create get_tags function which will return the tags for the title of the paper, citation , URL and author .

In [7]:
# this function for the extracting information of the tags
def get_tags(doc):
    paper_tag = doc.select('[data-lid]')
    cite_tag = doc.select('[title=Cite] + a')
    link_tag = doc.find_all('h3',{"class" : "gs_rt"})
    author_tag = doc.find_all("div", {"class": "gs_a"})

    return paper_tag,cite_tag,link_tag,author_tag

For extracting the title of the research paper , we will define the helper function get_papertitle which will use previously extracted paper tags list .

In [8]:
# it will return the title of the paper
def get_papertitle(paper_tag):
  
    paper_names = []
  
    for tag in paper_tag:
        paper_names.append(tag.select('h3')[0].get_text())

    return paper_names

Now get_citecount helper function defined for extracting information of citation count of the paper using above extracted cite_tag .If paper has no citation , then count will zero .

In [9]:
# it will return the number of citation of the paper
def get_citecount(cite_tag):
    cite_count = []
    for i in cite_tag:
        cite = i.text
    if i is None or cite is None:  # if paper has no citatation then consider 0
        cite_count.append(0)
    else:
        tmp = re.search(r'\d+', cite) # its handle the None type object error and re use to remove the string " cited by " and return only integer value
        if tmp is None :
            cite_count.append(0)
        else :
            cite_count.append(int(tmp.group()))

    return cite_count

The helper function get_link returns the URL of the paper.

In [10]:
# function for the getting link information
def get_link(link_tag):

    links = []

    for i in range(len(link_tag)) :
        links.append(link_tag[i].a['href']) 

    return links 

For extracting the authors name , published year of the paper and information of conference / journal of the paper in which paper published, we will define the function get_author_year_publi_info .



In [11]:
# function for the getting autho , year and publication information
def get_author_year_publi_info(authors_tag):
    years = []
    publication = []
    authors = []
    for i in range(len(authors_tag)):
        authortag_text = (authors_tag[i].text).split()
        year = int(re.search(r'\d+', authors_tag[i].text).group())
        years.append(year)
        publication.append(authortag_text[-1])
        author = authortag_text[0] + ' ' + re.sub(',','', authortag_text[1])
        authors.append(author)
  
    return years , publication, authors

Now , we have extract all the information of regarding the research papers of google scholar . We will store this information as dictionary .

In [12]:
import pandas as pd
# creating final repository
    
paper_repos_dict = {
                    'Paper Title' : [],
                    'Year' : [],
                    'Author' : [],
                   # 'Citation' : [],
                    'Publication' : [],
                    'Url of paper' : [] }

# adding information in repository
# def add_in_paper_repo(papername,year,author,cite,publi,link):
def add_in_paper_repo(papername,year,author, publi,link):
    paper_repos_dict['Paper Title'].extend(papername)
    paper_repos_dict['Year'].extend(year)
    paper_repos_dict['Author'].extend(author)
    #paper_repos_dict['Citation'].extend(cite)
    paper_repos_dict['Publication'].extend(publi)
    paper_repos_dict['Url of paper'].extend(link)

    return pd.DataFrame(paper_repos_dict)

In the above post , we will extract information from the single webpage . If we want to extract information from multiple page then we will iterate over the each page of google scholar. Here we are scraping the total 10 pages of data. We can scrape more .For this we need to change the number in the range.

In [13]:
import re
import time


for i in range (0,110,10):
    
    print("Analyzing Paper\n")
    
  # get url for the each page
  # url = "https://scholar.google.com/scholar?start={}&q=object+detection+in+aerial+image+&hl=en&as_sdt=0,5".format(i)
    #face recognition:
    url = "https://scholar.google.com/scholar?hl=it&as_sdt=0%2C5&q=face+recognition+system&oq=face+reco".format(i)

  # function for the get content of each page
    doc = get_paperinfo(url)

  # function for the collecting tags
    paper_tag,cite_tag,link_tag,author_tag = get_tags(doc)
  
  # paper title from each page
    papername = get_papertitle(paper_tag)

  # year , author , publication of the paper
    year , publication , author = get_author_year_publi_info(author_tag)

  # cite count of the paper 
    #cite = get_citecount(cite_tag)

  # url of the paper
    link = get_link(link_tag)
    
  # add in paper repo dict
   # final = add_in_paper_repo(papername,year,author,cite,publication,link)
    final = add_in_paper_repo(papername,year,author,publication,link)
    
  # #use sleep to avoid status code 429

    print("Time to sleep!\n")
    
    time.sleep(30)

Analyzing Paper

Time to sleep!



KeyboardInterrupt: 

Finally we will store information in the dataframe using pandas which will look like as :

In [26]:
final[:10]

NameError: name 'final' is not defined

In [27]:
final.to_csv('face_recognition_system.csv', sep=',', index=False,header=True)

NameError: name 'final' is not defined

In [ ]:
paper_repos_dict = {
                    'Paper Title' : [],
                    'Year' : [],
                    'Author' : [],
                   # 'Citation' : [],
                    'Publication' : [],
                    'Url of paper' : [] }

In [34]:
import re
import time

for i in range (0,110,10):
    
    print("Analyzing Paper\n")
    
  # get url for the each page
    
    #fusion method:
    url = "https://scholar.google.com/scholar?hl=it&as_sdt=0%2C5&q=fusion+methods&oq=fusion+method".format(i)

  # function for the get content of each page
    doc = get_paperinfo(url)

  # function for the collecting tags
    paper_tag,cite_tag,link_tag,author_tag = get_tags(doc)
  
  # paper title from each page
    papername = get_papertitle(paper_tag)

  # year , author , publication of the paper
    year , publication , author = get_author_year_publi_info(author_tag)

  # cite count of the paper 
    #cite = get_citecount(cite_tag)

  # url of the paper
    link = get_link(link_tag)
    
  # add in paper repo dict
   # final = add_in_paper_repo(papername,year,author,cite,publication,link)
    final1 = add_in_paper_repo(papername,year,author,publication,link)
    
  # #use sleep to avoid status code 429

    print("Time to sleep!\n")
    
    time.sleep(30)

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!



In [35]:
final1[:10]

,Paper Title,Year,Author,Publication,Url of paper


In [31]:
final1.to_csv('fusion_method.csv', sep=',', index=False,header=True)

NameError: name 'final1' is not defined

In [ ]:
paper_repos_dict = {
                    'Paper Title' : [],
                    'Year' : [],
                    'Author' : [],
                   # 'Citation' : [],
                    'Publication' : [],
                    'Url of paper' : [] }

In [13]:
import re
import time

for i in range (0,110,5):
    
    print("Analyzing Paper\n")
    
    #keypoint heatmap
    url = "https://scholar.google.com/scholar?hl=it&as_sdt=0%2C5&q=keypoint+heatmaps&oq=keypoint+heatmap".format(i)
    
  # function for the get content of each page
    doc = get_paperinfo(url)

  # function for the collecting tags
    paper_tag,cite_tag,link_tag,author_tag = get_tags(doc)
  
  # paper title from each page
    papername = get_papertitle(paper_tag)

  # year , author , publication of the paper
    year , publication , author = get_author_year_publi_info(author_tag)

  # cite count of the paper 
    #cite = get_citecount(cite_tag)

  # url of the paper
    link = get_link(link_tag)
    
  # add in paper repo dict
   # final = add_in_paper_repo(papername,year,author,cite,publication,link)
    final2 = add_in_paper_repo(papername,year,author,publication,link)
    
  # #use sleep to avoid status code 429

    print("Time to sleep!\n")
    
    time.sleep(30)

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!



In [14]:
final2[:10]

,Paper Title,Year,Author,Publication,Url of paper
0,Transpose: Keypoint localization via transformer,2021,S Yang,openaccess.thecvf.com,https://openaccess.thecvf.com/content/ICCV2021...
1,Transforming Temporal Embeddings to Keypoint H...,2022,F Negin,openaccess.thecvf.com,https://openaccess.thecvf.com/content/CVPR2022...
2,Starmap for category-agnostic keypoint and vie...,2018,X Zhou,openaccess.thecvf.com,http://openaccess.thecvf.com/content_ECCV_2018...
3,Semi-supervised keypoint localization,2021,O Moskvyak,arxiv.org,https://arxiv.org/abs/2101.07988
4,Bottom-up human pose estimation by ranking hea...,2020,K Sun,arxiv.org,https://arxiv.org/abs/2006.15480
5,[HTML][HTML] Detecting slender objects with un...,2021,Z Kong,Elsevier,https://www.sciencedirect.com/science/article/...
6,OSKDet: Orientation-Sensitive Keypoint Localiz...,2022,D Lu,openaccess.thecvf.com,https://openaccess.thecvf.com/content/CVPR2022...
7,Bottom-up human pose estimation via disentangl...,2021,Z Geng,openaccess.thecvf.com,http://openaccess.thecvf.com/content/CVPR2021/...
8,Centernet: Keypoint triplets for object detection,2019,K Duan,openaccess.thecvf.com,http://openaccess.thecvf.com/content_ICCV_2019...
9,Tokenpose: Learning keypoint tokens for human ...,2021,Y Li,openaccess.thecvf.com,http://openaccess.thecvf.com/content/ICCV2021/...


In [15]:
final2.to_csv('keypoint_heatmap.csv', sep=',', index=False,header=True)

In [16]:
paper_repos_dict = {
                    'Paper Title' : [],
                    'Year' : [],
                    'Author' : [],
                   # 'Citation' : [],
                    'Publication' : [],
                    'Url of paper' : [] }

In [17]:
for i in range (0,110,10):
    
    print("Analyzing Paper\n")
    
  # get url for the each page

    #facial neural aggregation network
    url = "https://scholar.google.com/scholar?hl=it&as_sdt=0%2C5&q=specific+facial+neural+aggregation+network&oq=specific+facial+neural+aggregation+ne".format(i)

  # function for the get content of each page
    doc = get_paperinfo(url)

  # function for the collecting tags
    paper_tag,cite_tag,link_tag,author_tag = get_tags(doc)
  
  # paper title from each page
    papername = get_papertitle(paper_tag)

  # year , author , publication of the paper
    year , publication , author = get_author_year_publi_info(author_tag)

  # cite count of the paper 
    #cite = get_citecount(cite_tag)

  # url of the paper
    link = get_link(link_tag)
    
  # add in paper repo dict
   # final = add_in_paper_repo(papername,year,author,cite,publication,link)
    final3 = add_in_paper_repo(papername,year,author,publication,link)
    
  # #use sleep to avoid status code 429

    print("Time to sleep!\n")
    
    time.sleep(30)

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!



In [18]:
final3[:10]

,Paper Title,Year,Author,Publication,Url of paper
0,Neural aggregation network for video face reco...,2017,J Yang,openaccess.thecvf.com,http://openaccess.thecvf.com/content_cvpr_2017...
1,Facial expression recognition using hierarchic...,2018,S Xie,ieeexplore.ieee.org,https://ieeexplore.ieee.org/abstract/document/...
2,Combining modality specific deep neural networ...,15,SE Kahou,dl.acm.org,https://dl.acm.org/doi/abs/10.1145/2522848.253...
3,Facial landmark detection with tweaked convolu...,2017,Y Wu,ieeexplore.ieee.org,https://ieeexplore.ieee.org/abstract/document/...
4,[HTML][HTML] Ordered weighted aggregation netw...,2021,J Rivero-Hernández,Elsevier,https://www.sciencedirect.com/science/article/...
5,Low quality video face recognition: Multi-mode...,2019,S Gong,openaccess.thecvf.com,http://openaccess.thecvf.com/content_ICCVW_201...
6,Style aggregated network for facial landmark d...,2018,X Dong,openaccess.thecvf.com,http://openaccess.thecvf.com/content_cvpr_2018...
7,Redundancy Removing Aggregation Network With D...,2020,Z Ou,ieeexplore.ieee.org,https://ieeexplore.ieee.org/abstract/document/...
8,Comparator networks,2018,W Xie,openaccess.thecvf.com,http://openaccess.thecvf.com/content_ECCV_2018...
9,Multi-prototype networks for unconstrained set...,2019,J Zhao,arxiv.org,https://arxiv.org/abs/1902.04755


In [19]:
final3.to_csv('neural_aggregation_network.csv', sep=',', index=False,header=True)

In [20]:
paper_repos_dict = {
                    'Paper Title' : [],
                    'Year' : [],
                    'Author' : [],
                   # 'Citation' : [],
                    'Publication' : [],
                    'Url of paper' : [] }

In [21]:
for i in range (0,110,10):
    
    print("Analyzing Paper\n")
    
  # get url for the each pagel = "https://scholar.google.com/scholar?hl=it&as_sdt=0%2C5&q=keypoint+heatmaps&oq=keypoint+heatmap".format(i)
    
    #face clustering
    url = "https://scholar.google.com/scholar?hl=it&as_sdt=0%2C5&q=face+clustering&oq=face+cluster".format(i)

  # function for the get content of each page
    doc = get_paperinfo(url)

  # function for the collecting tags
    paper_tag,cite_tag,link_tag,author_tag = get_tags(doc)
  
  # paper title from each page
    papername = get_papertitle(paper_tag)

  # year , author , publication of the paper
    year , publication , author = get_author_year_publi_info(author_tag)

  # cite count of the paper 
    #cite = get_citecount(cite_tag)

  # url of the paper
    link = get_link(link_tag)
    
  # add in paper repo dict
   # final = add_in_paper_repo(papername,year,author,cite,publication,link)
    final4 = add_in_paper_repo(papername,year,author,publication,link)
    
  # #use sleep to avoid status code 429

    print("Time to sleep!\n")
    
    time.sleep(30)

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!



In [22]:
final4[:10]

,Paper Title,Year,Author,Publication,Url of paper
0,[PDF][PDF] Hierarchical face clustering on pol...,2001,M Garland,dl.acm.org,https://dl.acm.org/doi/pdf/10.1145/364338.364345
1,Face clustering: representation and pairwise c...,2018,Y Shi,ieeexplore.ieee.org,https://ieeexplore.ieee.org/abstract/document/...
2,Constrained clustering and its application to ...,2013,B Wu,openaccess.thecvf.com,http://openaccess.thecvf.com/content_cvpr_2013...
3,Linkage based face clustering via graph convol...,2019,Z Wang,openaccess.thecvf.com,http://openaccess.thecvf.com/content_CVPR_2019...
4,Constrained multi-view video face clustering,2015,X Cao,ieeexplore.ieee.org,https://ieeexplore.ieee.org/abstract/document/...
5,Density-aware feature embedding for face clust...,2020,S Guo,openaccess.thecvf.com,http://openaccess.thecvf.com/content_CVPR_2020...
6,Robust face clustering via tensor decomposition,2014,X Cao,ieeexplore.ieee.org,https://ieeexplore.ieee.org/abstract/document/...
7,[HTML][HTML] Model simplification for meshing ...,2001,A Sheffer,Elsevier,https://www.sciencedirect.com/science/article/...
8,Self-supervised learning of face representatio...,2019,V Sharma,ieeexplore.ieee.org,https://ieeexplore.ieee.org/abstract/document/...
9,Video face clustering with unknown number of c...,2019,M Tapaswi,openaccess.thecvf.com,http://openaccess.thecvf.com/content_ICCV_2019...


In [23]:
final4.to_csv('face_cluster.csv', sep=',', index=False,header=True)

In [24]:
paper_repos_dict = {
                    'Paper Title' : [],
                    'Year' : [],
                    'Author' : [],
                   # 'Citation' : [],
                    'Publication' : [],
                    'Url of paper' : [] }

In [25]:

for i in range (0,110,10):
    
    print("Analyzing Paper\n")
    
  # get url for the each page

    #field view head mounted display
    url = "https://scholar.google.com/scholar?hl=it&as_sdt=0%2C5&q=field+view+head-mounted+display+base&btnG=".format(i)
    
    

  # function for the get content of each page
    doc = get_paperinfo(url)

  # function for the collecting tags
    paper_tag,cite_tag,link_tag,author_tag = get_tags(doc)
  
  # paper title from each page
    papername = get_papertitle(paper_tag)

  # year , author , publication of the paper
    year , publication , author = get_author_year_publi_info(author_tag)

  # cite count of the paper 
    #cite = get_citecount(cite_tag)

  # url of the paper
    link = get_link(link_tag)
    
  # add in paper repo dict
   # final = add_in_paper_repo(papername,year,author,cite,publication,link)
    final5 = add_in_paper_repo(papername,year,author,publication,link)
    
  # #use sleep to avoid status code 429

    print("Time to sleep!\n")
    
    time.sleep(30)

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!

Analyzing Paper

Time to sleep!



In [26]:
final5[:10]

,Paper Title,Year,Author,Publication,Url of paper
0,[LIBRO][B] Effects of field of view on perform...,2000,KW Arthur,search.proquest.com,https://search.proquest.com/openview/67103fd45...
1,Towards efficient visual guidance in limited f...,2018,F Bork,ieeexplore.ieee.org,https://ieeexplore.ieee.org/abstract/document/...
2,Augmenting the field-of-view of head-mounted d...,2016,R Xiao,dl.acm.org,https://dl.acm.org/doi/abs/10.1145/2858036.285...
3,Enabling focus cues in head-mounted displays,2017,H Hua,ieeexplore.ieee.org,https://ieeexplore.ieee.org/abstract/document/...
4,"Wide-angle, off-axis, see-through head-mounted...",2000,JP Rolland,spiedigitallibrary.org,https://www.spiedigitallibrary.org/journals/op...
5,[LIBRO][B] Head mounted displays,1997,JE Melzer,taylorfrancis.com,https://www.taylorfrancis.com/chapters/edit/10...
6,Multifocal planes head-mounted displays,2000,JP Rolland,opg.optica.org,https://opg.optica.org/abstract.cfm?uri=ao-39-...
7,A 3D integral imaging optical see-through head...,2014,H Hua,opg.optica.org,https://opg.optica.org/abstract.cfm?uri=oe-22-...
8,Using a large projection screen as an alternat...,2000,E Patrick,dl.acm.org,https://dl.acm.org/doi/abs/10.1145/332040.332479
9,Optical versus video see-through head-mounted ...,2000,JP Rolland,ieeexplore.ieee.org,https://ieeexplore.ieee.org/abstract/document/...


In [27]:
final5.to_csv('field_view_head_mounted_display.csv', sep=',', index=False,header=True)

In [28]:
paper_repos_dict = {
                    'Paper Title' : [],
                    'Year' : [],
                    'Author' : [],
                   # 'Citation' : [],
                    'Publication' : [],
                    'Url of paper' : [] }

In [ ]:
import re
import time

for i in range (0,110,10):
    
    print("Analyzing Paper\n")
    
  # get url for the each page

    #feature extraction network:
    url = "https://scholar.google.com/scholar?hl=it&as_sdt=0%2C5&q=feature+extraction+networks&oq=feature+extraction+network".format(i)

  # function for the get content of each page
    doc = get_paperinfo(url)

  # function for the collecting tags
    paper_tag,cite_tag,link_tag,author_tag = get_tags(doc)
  
  # paper title from each page
    papername = get_papertitle(paper_tag)

    print(papername)
  # year , author , publication of the paper
    year , publication , author = get_author_year_publi_info(author_tag)

  # cite count of the paper 
    #cite = get_citecount(cite_tag)

  # url of the paper
    link = get_link(link_tag)
    
  # add in paper repo dict
   # final = add_in_paper_repo(papername,year,author,cite,publication,link)
    final6 = add_in_paper_repo(papername,year,author,publication,link)
    
  # #use sleep to avoid status code 429

    print("Time to sleep!\n")
    
    time.sleep(30)

Analyzing Paper

[]
Time to sleep!



In [30]:
final6[:10]

,Paper Title,Year,Author,Publication,Url of paper
0,A mathematical theory of deep convolutional ne...,2017,T Wiatowski,ieeexplore.ieee.org,https://ieeexplore.ieee.org/abstract/document/...
1,Feature extraction and image recognition with ...,2018,YH Liu,iopscience.iop.org,https://iopscience.iop.org/article/10.1088/174...
2,Deep feature extraction and classification of ...,2016,Y Chen,ieeexplore.ieee.org,https://ieeexplore.ieee.org/abstract/document/...
3,Revisiting deep hyperspectral feature extracti...,2021,SK Roy,ieeexplore.ieee.org,https://ieeexplore.ieee.org/abstract/document/...
4,Decision boundary feature extraction for neura...,1997,C Lee,ieeexplore.ieee.org,https://ieeexplore.ieee.org/abstract/document/...
5,Artificial neural networks for feature extract...,1995,J Mao,ieeexplore.ieee.org,https://ieeexplore.ieee.org/abstract/document/...
6,Backbones-Review: Feature Extraction Networks ...,2022,O Elharrouss,arxiv.org,https://arxiv.org/abs/2206.08016
7,Feature extraction with deep neural networks b...,2012,A Stuhlsatz,ieeexplore.ieee.org,https://ieeexplore.ieee.org/abstract/document/...
8,"Deep convolutional neural networks: Structure,...",2017,I Namatēvs,itms-journals.rtu.lv,https://itms-journals.rtu.lv/article/view/itms...
9,Neuroscale: novel topographic feature extracti...,1996,D Lowe,proceedings.neurips.cc,https://proceedings.neurips.cc/paper/1996/hash...


In [32]:
final6.to_csv('feature_extraction_network.csv', sep=',', index=False,header=True)

In [16]:
paper_repos_dict = {
                    'Paper Title' : [],
                    'Year' : [],
                    'Author' : [],
                   # 'Citation' : [],
                    'Publication' : [],
                    'Url of paper' : [] }

In [17]:
import re
import time

for i in range (0,110,10):
    
    print("Analyzing Paper\n")
    
  # get url for the each page

    #feature extraction network:
    url = "https://scholar.google.com/scholar?hl=it&as_sdt=0%2C5&q=free+form+shape+face+region&btnG=".format(i)

  # function for the get content of each page
    doc = get_paperinfo(url)
    print(doc)

  # function for the collecting tags
    paper_tag,cite_tag,link_tag,author_tag = get_tags(doc)
  
  # paper title from each page
    papername = get_papertitle(paper_tag)
    print(papername)

  # year , author , publication of the paper
    year , publication , author = get_author_year_publi_info(author_tag)

  # cite count of the paper 
    #cite = get_citecount(cite_tag)

  # url of the paper
    link = get_link(link_tag)
    
  # add in paper repo dict
   # final = add_in_paper_repo(papername,year,author,cite,publication,link)
    final7 = add_in_paper_repo(papername,year,author,publication,link)
    
  # #use sleep to avoid status code 429

    print("Time to sleep!\n")
    
    time.sleep(30)

Analyzing Paper

<!DOCTYPE html>
<html><head><title>Google Scholar</title><meta content="text/html;charset=utf-8" http-equiv="Content-Type"/><meta content="IE=Edge" http-equiv="X-UA-Compatible"/><meta content="origin-when-cross-origin" name="referrer"/><meta content="width=device-width,initial-scale=1,minimum-scale=1,maximum-scale=2" name="viewport"/><meta content="telephone=no" name="format-detection"/><link href="/favicon.ico" rel="shortcut icon"/><style>html,body,form,table,div,h1,h2,h3,h4,h5,h6,img,ol,ul,li,button{margin:0;padding:0;border:0;}table{border-collapse:collapse;border-width:0;empty-cells:show;}html,body{height:100%}#gs_top{position:relative;box-sizing:border-box;min-height:100%;min-width:964px;-webkit-tap-highlight-color:rgba(0,0,0,0);}#gs_top>*:not(#x){-webkit-tap-highlight-color:rgba(204,204,204,.5);}.gs_el_ph #gs_top,.gs_el_ta #gs_top{min-width:320px;}#gs_top.gs_nscl{position:fixed;width:100%;}body,td,input,button{font-size:13px;font-family:Arial,sans-serif;line-heig

KeyboardInterrupt: 

In [16]:
final7[:10]

,Paper Title,Year,Author,Publication,Url of paper


In [18]:
final7.to_csv('free_form_shape_face_region.csv', sep=',', index=False,header=True)